## Project Kijkcijfers
Onderzoeken welke facturen de kijkcijfers beïnvloeden alsook kijkcijfers voorspellen.

### Stappenplan
1. Probleemstelling
2. Data gathering
3. Data visualiseren en analyseren
4. Data cleaning
5. Model selection and training
6. Fine tune model
6. Launch & maintain

# Probleemstelling
Onderzoeken welke facturen de kijkcijfers beïnvloeden alsook kijkcijfers voorspellen.

# Data gathering
Data van de kijkcijfers is de vinden op CIM. We maken een klein python scriptje die iteratief de data van via API calls ophaalt bij CIM en vervolgens opslaat in een csv file. zie 'gather-views.py'

1ste probleem: data voor 14 en 15 augustus 2019 ontbreken. We passen het script aan zodat het niet stopt als er 1 dag geen data gevonden wordt, maar pas stopt als het 5 achtereenvolgende dagen geen data meer kan ophalen.

Data wordt opgeslaan in 'tv_views_data.csv'

Sidenote: Ik heb er aan gedacht om de weersvoorspelling toe te voegen aan deze datalijst maar in mijn opinie zorgt dit voor een extra probabiliteit en hebben we meer info door gebruik te maken van seizoenen. Uit mijn ervaring is de weersvoorspelling in België een extra factor van onzekerheid en daarom geen goede maatstaf om toe te voegen aan het model.

# Data visualisatie
Hier gaan we de data uit de csv visualiseren en onderzoeken. Kijken waar we bepaalde zaken kunnen opkuisen en eventueel al enkele patronen kunnen opmerken.

Hieronder importeren we eerst enkele belangrijke packages alsook een check van de python versie zoals we in elke les deden.

In [2]:
# Python ≥3.11 is required
import sys
assert sys.version_info >= (3, 11)

# Scikit-Learn ≥ 1.6 is required
from packaging import version
import sklearn
print(sklearn.__version__)
assert version.parse(sklearn.__version__) >= version.parse("1.6")

# Common imports
import numpy as np
import pandas as pd
import os

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt

1.6.1


Als we de info van de lijst bekijken zien we dat er heel wat null values zijn. Ik wil eerst alle data bekijken alvorens ik dit ga opkuisen. Zo krijg ik al een eerste visueel beeld van de data alvorens ik vanalles ga verwijderen. Ookal ben ik vrijwel zeker dat de null values weg mogen.

In [68]:
tv_views = pd.read_csv('tv_views_data.csv')
# alle kolomen goedzetten
tv_views = tv_views.dropna(axis=1, how='all')
# Convert data types
tv_views['description'] = tv_views['description'].astype(str)
tv_views['channel'] = tv_views['channel'].astype(str)
tv_views['live'] = tv_views['live'].astype(bool)
tv_views['rateInK'] = pd.to_numeric(tv_views['rateInK'], errors='coerce')

# Convert rLength to timedelta
tv_views['rLength'] = pd.to_timedelta(tv_views['rLength'], errors='coerce')

tv_views = tv_views.dropna(axis=0, subset=['startTime', 'rLength', 'rateInK'])

tv_views.info()
tv_views.head()

<class 'pandas.core.frame.DataFrame'>
Index: 58828 entries, 0 to 62458
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   description  58828 non-null  object         
 1   channel      58828 non-null  object         
 2   startTime    58828 non-null  object         
 3   rLength      58828 non-null  timedelta64[ns]
 4   rateInK      58828 non-null  float64        
 5   live         58828 non-null  bool           
 6   date         58828 non-null  object         
dtypes: bool(1), float64(1), object(4), timedelta64[ns](1)
memory usage: 3.2+ MB


,description,channel,startTime,rLength,rateInK,live,date
0,THUIS,VRT 1,20:24:14,0 days 00:24:08,873.935,False,2025-04-03
1,HET 7 UUR-JOURNAAL,VRT 1,19:00:05,0 days 00:48:22,859.904,False,2025-04-03
2,MAN BIJT HOND,VRT 1,19:51:09,0 days 00:24:55,702.872,False,2025-04-03
3,NIEUWS 19U VTM,VTM,18:59:50,0 days 00:59:10,562.650,False,2025-04-03
4,BLOKKEN,VRT 1,18:29:28,0 days 00:28:30,478.774,False,2025-04-03


In [70]:
# transform channel to category
tv_views['channel'].value_counts()
channel_mapping = {
    'EEN': '1',
    'VTM': '3',
    'VRT 1': '1',
    'VIER': '4',
    'PLAY4': '4',
    'Canvas': '2',
    'CANVAS': '2',
    'VRT CANVAS': '2',
    'Q2': '6',
    'VTM2': '6',
    'VITAYA': '7',
    'PLAY5': '5',
    'VIJF1': '5',
    'VTM3': '7',
    'EUROSPORT 1 (NL)': '11',
    'OP 12': '2',
    'VTM4': '8',
    'VTM GOLD': '18',
    'PLAY SPORTS OPEN': '11',
    'TF1': '99',
    'ZES': '6',
    'AB3': '99',
    'Source': '99',
    'All': '99',
    'Channel': '99',
    'KETNET': '2',
    'ELEVEN PRO LEAGUE 1 NL': '11',
    'nan': '99'
}
# First map the channels
tv_views['channel'] = tv_views['channel'].map(channel_mapping)

# Then remove rows where channel is NaN (no mapping found)
tv_views = tv_views.dropna(subset=['channel'])

# Convert to integer
tv_views['channel'] = tv_views['channel'].astype(int)

# Display the first few rows to verify
display(tv_views.head())




,description,channel,startTime,rLength,rateInK,live,date


In [71]:
# histogram maken van alle waarden per dag => overzicht scheppen
tv_views['description'].value_counts()

Series([], Name: count, dtype: int64)

In [ ]:
# transform description to category


In [51]:

# transform starttime to number per halfhour
# Convert startTime to numeric value (0.5 per half hour)
def time_to_numeric(time_str):
    # Handle float values by converting to string first
    if isinstance(time_str, float):
        return time_str
    try:
        hours, minutes, seconds = map(int, time_str.split(':'))
        total_hours = hours + minutes/60 + seconds/3600
        return round(total_hours * 2) / 2  # Round to nearest 0.5
    except (ValueError, AttributeError):
        return time_str  # Return original value if conversion fails

tv_views['startTime'] = tv_views['startTime'].apply(time_to_numeric)



In [52]:
# transform rlength to seconds
tv_views['rLength'] = tv_views['rLength'].dt.total_seconds().astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

Eerst doen we een algemene visualisatie van de data. Daarna wil ook wat dieper ingaan op de data rond de Covid-19 lockdown periodes. Volgens mij zouden de kijkcijfers daar iets hoger liggen dan het gemiddelde. Afhankelijk van deze observatie moet ik de beslissing maken of ik deze data als uitschieter benoem of niet. Want laat ons hopen dat zo'n lockdown niet meer terugkomt.

Periode waar horeca gesloten was:
- 13 maart 2020 tot 8 juni 2020
- 16 oktober 2020 tot 11 mei 2021

In [47]:
# check viewer count per day to see the effect of the lockdowns
tv_views.head()


,description,channel,startTime,rLength,rateInK,live,date,startTime_numeric
0,THUIS,1,20.5,0,873.935,False,2025-04-03,20.5
1,HET 7 UUR-JOURNAAL,1,19.0,0,859.904,False,2025-04-03,19.0
2,MAN BIJT HOND,1,20.0,0,702.872,False,2025-04-03,20.0
3,NIEUWS 19U VTM,3,19.0,0,562.650,False,2025-04-03,19.0
4,BLOKKEN,1,18.5,0,478.774,False,2025-04-03,18.5
